In [1]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching
from utils.deformable_icp import DeformableICP

In [ ]:
OUTPUT_DIR = '../OUTPUT'
DATA_DIR = './pa345_data'

RIGID_BODY_DATA_A = f'{DATA_DIR}/Problem5-BodyA.txt'
RIGID_BODY_DATA_B = f'{DATA_DIR}/Problem5-BodyB.txt'

SURFACE_DATA = f'{DATA_DIR}/Problem5MeshFile.sur'

SAMPLE_ID = 'F'
SAMPLE_DATA = f'./pa345_data/PA5-{SAMPLE_ID}-Debug-SampleReadingsTest.txt'
# SAMPLE_DATA = f'./pa345_data/PA5-{SAMPLE_ID}-Unknown-SampleReadingsTest.txt'

# Load data files
rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)

modes_dl = AtlasModesDataloader.read_file(f'{DATA_DIR}/Problem5Modes.txt')
print(modes_dl.modes.shape)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [3]:
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg = PointCloudRegistration(verbose=False)
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

### Get d<sub>k</sub>

In [ ]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)

### Compute s<sub>k</sub>, c<sub>k</sub>

In [ ]:
import csv
import time

csv_elapsed_time_file = f'{OUTPUT_DIR}/pa5-ElapsedTime.csv'
with open(csv_elapsed_time_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Iteration', 'Elapsed Time (s)'])


for i in range(1):
    start_time = time.time()

    # Initialize ICP helper class 
    deform_icp = DeformableICP()

    # Initialize meshgrid of Triangles
    mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)

    best_pt_cloud, closest_pt, dist, F_best, λ_best = deform_icp(d_k, mesh, modes_dl.modes)

    elapsed_time = time.time() - start_time

    with open(csv_elapsed_time_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, elapsed_time])

In [ ]:
print(λ_best)

### write output to file

In [ ]:
output_file = f'{OUTPUT_DIR}/pa5-{SAMPLE_ID}-Output.txt'
with open(output_file, 'w') as file:
    file.write(f"{num_samples}, {output_file}\n")
    file.write(" ".join(map(lambda x: f"{x:.4f}", λ_best[1:])) + "\n")
    for sample in range(num_samples):
        file.write(f"{best_pt_cloud[sample][0]:.2f} {best_pt_cloud[sample][1]:.2f} {best_pt_cloud[sample][2]:.2f} ")
        file.write(f"{closest_pt[sample][0]:.2f} {closest_pt[sample][1]:.2f} {closest_pt[sample][2]:.2f}")
        file.write(f" {dist[sample]:.2f}")
        file.write("\n")

print(f"Output written to {output_file}")


### Compare predicted with debug output

In [ ]:
import csv

csv_output_file = f'{OUTPUT_DIR}/pa5-ErrorMetrics.csv'
with open(csv_output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sample', "Error_weights", 'Error_s_k', 'Error_c_k', 'Error_norm'])

for sample_id in ['A', 'B', 'C', 'D', 'E', 'F']:
    SAMPLE_ID = sample_id

    pred_file = f'{OUTPUT_DIR}/pa5-{SAMPLE_ID}-Output.txt'
    gt_file = f'{DATA_DIR}/PA5-{SAMPLE_ID}-Debug-Answer.txt'

    dl1 = OutputDataloader.read_file(pred_file)
    dl2 = OutputDataloader.read_file(gt_file)

    weights1 = dl1.weights
    weights2 = dl2.weights

    raw_data1 = dl1.raw_data
    raw_data2 = dl2.raw_data

    error_weights = np.mean(np.abs(weights1 - weights2))

    # get the mean absolute error for d_x
    error_s_x = np.mean(np.abs(raw_data1[:, 0] - raw_data2[:, 0]))
    # get the mean absolute error for d_y
    error_s_y = np.mean(np.abs(raw_data1[:, 1] - raw_data2[:, 1]))
    # get the mean absolute error for d_z
    error_s_z = np.mean(np.abs(raw_data1[:, 2] - raw_data2[:, 2]))
    error_s_k = (error_s_x, error_s_y, error_s_z)
    error_s_k_avg = np.mean(error_s_k)

    # get the mean absolute error for c_x
    error_c_x = np.mean(np.abs(raw_data1[:, 3] - raw_data2[:, 3]))
    # get the mean absolute error for c_y
    error_c_y = np.mean(np.abs(raw_data1[:, 4] - raw_data2[:, 4]))
    # get the mean absolute error for c_z
    error_c_z = np.mean(np.abs(raw_data1[:, 5] - raw_data2[:, 5]))
    error_c_k = (error_c_x, error_c_y, error_c_z)
    error_c_k_avg = np.mean(error_c_k)

    # get the mean absolute error for ||d_k-c_k||
    error_norm = np.mean(np.abs(raw_data1[:, 6] - raw_data2[:, 6]))
    with open(csv_output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([SAMPLE_ID, round(error_weights, 4), round(error_s_k_avg, 4), round(error_c_k_avg, 4), round(error_norm, 4)])

print(f"Error metrics written to {csv_output_file}")

### Unknown output summary

In [ ]:
import csv

csv_output_file = f'{OUTPUT_DIR}/pa5-Unknown-Summary.csv'
with open(csv_output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sample', "Average weight", 'Average s_k', 'Average c_k', 'Average norm'])

for sample_id in ['G', 'H', 'J', 'K']:
    SAMPLE_ID = sample_id

    pred_file = f'{OUTPUT_DIR}/pa5-{SAMPLE_ID}-Output.txt'

    dl1 = OutputDataloader.read_file(pred_file)

    weights1 = dl1.weights

    raw_data1 = dl1.raw_data

    error_weights = np.mean(np.abs(weights1))

    # get the mean absolute error for d_x
    error_s_x = np.mean(np.abs(raw_data1[:, 0]))
    # get the mean absolute error for d_y
    error_s_y = np.mean(np.abs(raw_data1[:, 1]))
    # get the mean absolute error for d_z
    error_s_z = np.mean(np.abs(raw_data1[:, 2]))
    error_s_k = (error_s_x, error_s_y, error_s_z)
    error_s_k_avg = np.mean(error_s_k)

    # get the mean absolute error for c_x
    error_c_x = np.mean(np.abs(raw_data1[:, 3]))
    # get the mean absolute error for c_y
    error_c_y = np.mean(np.abs(raw_data1[:, 4]))
    # get the mean absolute error for c_z
    error_c_z = np.mean(np.abs(raw_data1[:, 5]))
    error_c_k = (error_c_x, error_c_y, error_c_z)
    error_c_k_avg = np.mean(error_c_k)

    # get the mean absolute error for ||d_k-c_k||
    error_norm = np.mean(np.abs(np.subtract(error_s_k, error_c_k)))
    with open(csv_output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([SAMPLE_ID, round(error_weights, 4), round(error_s_k_avg, 4), round(error_c_k_avg, 4), round(error_norm, 4)])

print(f"Error metrics written to {csv_output_file}")